<a href="https://colab.research.google.com/github/crodier1/machine_learning_deep_learning/blob/main/Koine_Greek_Tutor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Read this first!

Make sure you get your Open AI API key first. Put it under secrets with the name "OPENAI_API_KEY".

In [1]:
!apt-get install -y -q python3-dev libasound2-dev
!pip install -q simpleaudio

Reading package lists...
Building dependency tree...
Reading state information...
libasound2-dev is already the newest version (1.2.6.1-1ubuntu1).
python3-dev is already the newest version (3.10.6-1~22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


In [2]:
import os
import json
from google.colab import userdata
from openai import OpenAI
import gradio as gr
from pydub import AudioSegment
from io import BytesIO
import tempfile
import simpleaudio as sa
import threading

In [3]:
open_api_key = userdata.get('OPENAI_API_KEY')

openai = OpenAI(api_key=open_api_key)
MODEL = 'gpt-4o-mini'

In [4]:
system_message = (
    "You are a Koine Greek tutor. Your role is to help the user understand New Testament verses "
    "in their original Koine Greek. "
    "Begin by presenting a verse in Koine Greek and ask the user to translate it into English. "
    "If the user's translation is incorrect, gently explain the mistake, highlighting the relevant grammar or vocabulary, "
    "then provide another verse of similar or slightly easier difficulty. "
    "If the user's translation is correct, congratulate them, briefly reinforce why it is correct, "
    "and then provide a verse of equal or slightly greater difficulty. "
    "Gradually increase the complexity of verses as the user demonstrates accuracy, moving from simple sentences "
    "to more complex syntax (e.g., participial phrases, subordinate clauses, theological vocabulary). "
    "Occasionally ask the user to explain the meaning of a single word, parsing of a verb, or the function of a case ending, "
    "not just the full translation. "
    "Always keep the tone encouraging, interactive, and focused on learning. "
    "Never provide the English translation until after the user attempts their own."
)


In [5]:
famous_nt_verses = [
    {"John 3:16": "Οὕτως γὰρ ἠγάπησεν ὁ Θεὸς τὸν κόσμον, ὥστε τὸν Υἱὸν αὐτοῦ τὸν μονογενῆ ἔδωκεν, ἵνα πᾶς ὁ πιστεύων εἰς αὐτὸν μὴ ἀπόληται ἀλλ᾽ ἔχῃ ζωὴν αἰώνιον."},

    {"Matthew 5:9": "Μακάριοι οἱ εἰρηνοποιοί, ὅτι αὐτοὶ υἱοὶ Θεοῦ κληθήσονται."},

    {"Romans 8:28": "Οἴδαμεν δὲ ὅτι τοῖς ἀγαπῶσιν τὸν Θεὸν πάντα συνεργεῖ εἰς ἀγαθόν, τοῖς κατὰ πρόθεσιν κλητοῖς οὖσιν."},

    {"Philippians 4:13": "Πάντα ἰσχύω ἐν τῷ ἐνδυναμοῦντί με Χριστῷ."},

    {"1 Corinthians 13:13": "νυνὶ δὲ μένει πίστις, ἐλπίς, ἀγάπη, τὰ τρία ταῦτα· μείζων δὲ τούτων ἡ ἀγάπη."},

    # The Lord's Prayer (Matthew 6:9–13)
    {"Matthew 6:9": "Πάτερ ἡμῶν ὁ ἐν τοῖς οὐρανοῖς, ἁγιασθήτω τὸ ὄνομά σου·"},
    {"Matthew 6:10": "ἐλθέτω ἡ βασιλεία σου· γενηθήτω τὸ θέλημά σου, ὡς ἐν οὐρανῷ καὶ ἐπὶ γῆς·"},
    {"Matthew 6:11": "τὸν ἄρτον ἡμῶν τὸν ἐπιούσιον δὸς ἡμῖν σήμερον·"},
    {"Matthew 6:12": "καὶ ἄφες ἡμῖν τὰ ὀφειλήματα ἡμῶν, ὡς καὶ ἡμεῖς ἀφήκαμεν τοῖς ὀφειλέταις ἡμῶν·"},
    {"Matthew 6:13": "καὶ μὴ εἰσενέγκῃς ἡμᾶς εἰς πειρασμόν, ἀλλὰ ῥῦσαι ἡμᾶς ἀπὸ τοῦ πονηροῦ."},

    # John 1:1–5
    {"John 1:1": "Ἐν ἀρχῇ ἦν ὁ Λόγος, καὶ ὁ Λόγος ἦν πρὸς τὸν Θεόν, καὶ Θεὸς ἦν ὁ Λόγος."},
    {"John 1:2": "Οὗτος ἦν ἐν ἀρχῇ πρὸς τὸν Θεόν."},
    {"John 1:3": "πάντα δι᾽ αὐτοῦ ἐγένετο, καὶ χωρὶς αὐτοῦ ἐγένετο οὐδὲ ἕν. ὃ γέγονεν"},
    {"John 1:4": "ἐν αὐτῷ ζωὴ ἦν, καὶ ἡ ζωὴ ἦν τὸ φῶς τῶν ἀνθρώπων·"},
    {"John 1:5": "καὶ τὸ φῶς ἐν τῇ σκοτίᾳ φαίνει, καὶ ἡ σκοτία αὐτὸ οὐ κατέλαβεν."}
]


In [6]:
import random

def get_verse():
    return random.choice(famous_nt_verses)

In [7]:
verse_function = {
    "name": "get_verse",
    "description": "Return one NT verse in Koine Greek from the famous_nt_verses list.",
    "parameters": {
        "type": "object",
        "properties": {}
    }
}

In [8]:
tools = [{"type": "function", "function": verse_function}]

In [9]:
from io import BytesIO
from pydub import AudioSegment
# 💡 Import the necessary display tools
from IPython.display import Audio, display

# The play_audio function is no longer needed

def talker(message):
    response = openai.audio.speech.create(
        model="tts-1",
        voice="onyx",
        input=message
    )

    # 1. Get the raw audio content (MP3 format)
    audio_content = response.content

    # 2. Use IPython.display.Audio to create an in-notebook player
    # Pass the raw content and specify the format so the browser knows how to play it.
    # The default sampling rate for OpenAI TTS is 24000 Hz.
    display(Audio(audio_content, rate=24000, autoplay=True))

# Example usage:
# talker("Hello how are you? This audio will play directly in your browser.")

In [10]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=tools
    )

    choice = response.choices[0].message

    # If the model calls the function
    if choice.tool_calls:
        for tool_call in choice.tool_calls:
            if tool_call.function.name == "get_verse":
                verse = get_verse()
                verse_text = f"Please translate this: {list(verse.keys())[0]}: {list(verse.values())[0]}"
                threading.Thread(target=talker, args=(verse_text,), daemon=True).start()
                return verse_text

    # Otherwise, normal assistant content
    threading.Thread(target=talker, args=(choice.content,), daemon=True).start()
    return choice.content


In [ ]:
opening_message = "Hi, I'm Ioannes, your Koine Greek tutor. Are you ready to start?"

initial_messages = [
    {"role": "assistant", "content": opening_message}
]

with gr.Blocks() as demo:
    chatbot = gr.Chatbot(value=initial_messages, type="messages")
    chat_interface = gr.ChatInterface(fn=chat, type="messages", chatbot=chatbot)

demo.launch(debug=True)

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://e3b22d404e75dcc124.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
